# Advanced Diagnostic Plots

## Detailed Analysis

This script is used to generate high-level diagnostic figures that validate both the physical consistency of the dataset and the spatial performance of the machine learning models.

The script performs two distinct tasks:
1.  **Universal Relations Check:** It verifies whether the generated data adheres to known astrophysical constraints, specifically the relationship between Compactness and Tidal Deformability (related to I-Love-Q relations).
2.  **Misclassification Mapping:** It creates a "Geography of Failure" visualization. This maps exactly where in the Mass-Radius phase space the machine learning models make errors, and how those errors diminish as more physical features are added to the classifiers.

## Physics and Math

### Compactness
The compactness parameter $C$ characterizes how dense an object is relative to a black hole. It is a dimensionless quantity defined as the ratio of the gravitational radius to the actual radius:

$$
C = \frac{G M}{R c^2}
$$

In the code, the constant `A_CONV` represents $\frac{G M_{\odot}}{c^2}$ (approximately 1.477 km). The compactness is computed as:

$$
C = \text{A\_CONV} \times \frac{M \, [M_{\odot}]}{R \, [\text{km}]}
$$

### Physical Limits
Two theoretical limits are visualized to frame the data:
1.  **Black Hole Limit:** $C = 0.5$ (The Schwarzschild radius is $2M$, so $R=2M \implies C=0.5$).
2.  **Buchdahl Limit:** $C = 4/9 \approx 0.444$. This is the maximum theoretical compactness for a static sphere of fluid in General Relativity before it must collapse.

## Code Walkthrough

### 1. Universal Relations Plotting
The function `plot_universal_relations` is used to examine the correlation between Compactness and Tidal Deformability.

```python
# Calculate Compactness
df['Compactness'] = CONSTANTS['A_CONV'] * (df['Mass'] / df['Radius'])
```

The data is subsampled for performance, and Hadronic vs. Quark stars are plotted on the $C - \log_{10}(\Lambda)$ plane. This visualization confirms that as stars become more compact (higher $C$), their tidal deformability decreases (lower $\Lambda$), following a tight "universal" track regardless of the specific EoS.

### 2. Misclassification Mapping
The function `plot_misclassification_map` visualizes how the decision boundaries of the models evolve. It iterates through the model hierarchy (`Geo` $\rightarrow$ `D`).

**Prediction and Masking:**
For each model, predictions are generated on the test set. Boolean masks are created to identify four categories of points:
*   **Correct Hadronic:** $y_{true}=0, y_{pred}=0$
*   **Correct Quark:** $y_{true}=1, y_{pred}=1$
*   **False Positive (FP):** Hadronic stars classified as Quark.
*   **False Negative (FN):** Quark stars classified as Hadronic.

**Layered Plotting:**
A 2x3 grid of subplots is created. In each panel:
1.  **Background:** Correct predictions are plotted as faint, small dots. This shows the general distribution of the test set.
2.  **Foreground:** Errors (FP and FN) are plotted as large 'X' markers.

```python
# Plot Errors (Foreground)
ax.scatter(X_test_all.loc[mask_fp, 'Radius'], ..., marker='x', c='#004488', label='False Pos')
ax.scatter(X_test_all.loc[mask_fn, 'Radius'], ..., marker='x', c='#DDAA33', label='False Neg')
```

This approach highlights *where* the models are confused. For simpler models (like 'Geo'), errors are concentrated in the region where Hadronic and Quark Mass-Radius curves overlap. As features like Tidal Deformability (Model A) and Slope (Model D) are added, these error clusters shrink or disappear.

## Visualization Output

### Figure 1: Universal Relations
*   **X-axis:** Compactness $C$.
*   **Y-axis:** Log Tidal Deformability $\log_{10}\Lambda$.
*   **Content:** A scatter plot showing the inverse relationship between compactness and deformability. Vertical lines mark the Buchdahl limit and the Black Hole limit. This serves as a sanity check that the generated EoS curves obey General Relativity.

### Figure 2: Misclassification Map (5-Panel Grid)
*   **Grid Layout:** 2 rows, 3 columns (one per model: Geo, A, B, C, D).
*   **Axes:** Mass vs. Radius (standard M-R plot).
*   **Markers:**
    *   **Faint Dots:** Correctly classified stars.
    *   **Blue 'X':** False Positives (Hadronic mistaken for Quark).
    *   **Yellow 'X':** False Negatives (Quark mistaken for Hadronic).
*   **Observation:** This figure demonstrates the "Geography of Failure." It shows visually how adding physics features clears up ambiguity in specific regions of the Mass-Radius plane (e.g., the confusion region around $1.4 M_{\odot}$ and $11-13$ km).